In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_Cancer"
cohort = "GSE218438"

# Input paths
in_trait_dir = "../../input/GEO/Liver_Cancer"
in_cohort_dir = "../../input/GEO/Liver_Cancer/GSE218438"

# Output paths
out_data_file = "../../output/preprocess/Liver_Cancer/GSE218438.csv"
out_gene_data_file = "../../output/preprocess/Liver_Cancer/gene_data/GSE218438.csv"
out_clinical_data_file = "../../output/preprocess/Liver_Cancer/clinical_data/GSE218438.csv"
json_path = "../../output/preprocess/Liver_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Structure Activity Relationship Read Across and Transcriptomics for Branched Carboxylic Acids"
!Series_summary	"The purpose of this study was to use chemical similarity evaluations, transcriptional profiling, in vitro toxicokinetic data and physiologically based pharmacokinetic (PBPK) models to support read across for a series of branched carboxylic acids using valproic acid (VPA), a known developmental toxicant, as a comparator. The chemicals included 2-propylpentanoic acid (VPA), 2-ethylbutanoic acid (EBA), 2-ethylhexanoic acid (EHA), 2-methylnonanoic acid (MNA), 2-hexyldecanoic acid (HDA), 2-propylnonanoic acid (PNA), dipentyl acetic acid (DPA) or 2-pentylheptanoic acid (PHA), octanoic acid (OA, a straight chain alkyl acid) and 2-ethylhexanol. Transcriptomics was evaluated in four cell types (A549, HepG2, MCF7 and iCell cardiomyocytes) 6 hours after exposure to 3 concentrations of the compounds, using the L1000 platform. The transcriptional pro

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains transcriptomic data from four cell types
# However, it appears to be a study focused on chemicals/compounds rather than human disease/traits
is_gene_available = True  # The dataset contains gene expression data (L1000 platform)

# 2. Variable Availability and Data Type Conversion
# From the sample characteristics dictionary, we only have cell type information
# No information about liver cancer (our trait), age, or gender

# 2.1 Data Availability
trait_row = None  # No liver cancer trait information available
age_row = None    # No age information available
gender_row = None # No gender information available

# 2.2 Data Type Conversion functions
# Even though we don't have the data, we'll define conversion functions as required
def convert_trait(value):
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if value in ['liver cancer', 'hepatocellular carcinoma', 'hcc', 'yes', 'true', 'positive']:
        return 1
    elif value in ['normal', 'control', 'no', 'false', 'negative']:
        return 0
    return None

def convert_age(value):
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    else:
        value = value.strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if value in ['male', 'm', 'man']:
        return 1
    elif value in ['female', 'f', 'woman']:
        return 0
    return None

# 3. Save Metadata
# Conduct initial filtering on the usability of the dataset
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. No need to extract clinical features since trait_row is None (clinical data not available for our trait of interest)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Liver_Cancer/GSE218438/GSE218438_series_matrix.txt.gz


Gene data shape: (22268, 636)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# These identifiers (e.g., '1007_s_at', '1053_at') appear to be Affymetrix probe IDs, not human gene symbols
# Affymetrix probe IDs need to be mapped to human gene symbols for biological interpretation
# Therefore, gene mapping is required

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'FLAG', 'SEQUENCE', 'SPOT_ID']
{'ID': ['1007_s_at', '121_at', '200024_at', '200045_at', '200053_at'], 'FLAG': ['LM', 'LM', 'LM', 'LM', 'LM'], 'SEQUENCE': ['GCTTCTTCCTCCTCCATCACCTGAAACACTGGACCTGGGG', 'TGTGCTTCCTGCAGCTCACGCCCACCAGCTACTGAAGGGA', 'ATGCCTTCGAGATCATACACCTGCTCACAGGCGAGAACCC', 'GGTGGTGCTGTTCTTTTCTGGTGGATTTAATGCTGACTCA', 'TGCTATTAGAGCCCATCCTGGAGCCCCACCTCTGAACCAC'], 'SPOT_ID': ['1007_s_at', '121_at', '200024_at', '200045_at', '200053_at']}

Examining potential gene mapping columns:


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify relevant columns for mapping
print("Looking for gene symbol information in the SOFT file...")

# Try to find platform annotation data with gene symbols
gene_symbols_found = False

try:
    # First attempt: read platform data more comprehensively
    with gzip.open(soft_file, 'rt') as f:
        # Look for platform annotation sections
        platform_section = False
        header_line = None
        annotation_lines = []
        
        for line in f:
            if line.startswith('!Platform_table_begin'):
                platform_section = True
                continue
            elif line.startswith('!Platform_table_end'):
                break
            
            if platform_section:
                if header_line is None:
                    header_line = line.strip()
                else:
                    annotation_lines.append(line.strip())
        
        if header_line:
            # Create a dataframe from the platform annotation
            headers = header_line.split('\t')
            platform_data = pd.DataFrame([line.split('\t') for line in annotation_lines], columns=headers)
            
            # Check for columns that might contain gene symbols
            gene_symbol_columns = [col for col in platform_data.columns if 'gene' in col.lower() and 'symbol' in col.lower()]
            gene_title_columns = [col for col in platform_data.columns if 'gene' in col.lower() and 'title' in col.lower()]
            
            potential_columns = gene_symbol_columns + gene_title_columns
            
            if potential_columns:
                print(f"Found potential gene symbol columns: {potential_columns}")
                # Use the first suitable column for mapping
                gene_column = potential_columns[0]
                platform_data = platform_data[['ID', gene_column]]
                platform_data.columns = ['ID', 'Gene']
                mapping_df = platform_data
                gene_symbols_found = True
                print(f"Using column '{gene_column}' for gene symbols")
                print("Sample of mapping:")
                print(mapping_df.head())
            else:
                print("No gene symbol columns found in platform data")
        else:
            print("Could not parse platform annotation data")
            
except Exception as e:
    print(f"Error processing platform data: {e}")

# If we couldn't find gene symbols, create a simple mapping as fallback
if not gene_symbols_found:
    print("Creating fallback mapping using probe IDs as pseudo-genes")
    # Create a basic mapping where each probe ID is treated as a separate gene
    mapping_df = pd.DataFrame({
        'ID': gene_data.index.tolist(),
        'Gene': gene_data.index.tolist()  # Just use probe IDs as pseudo-genes
    })
    print("Sample of fallback mapping:")
    print(mapping_df.head())

# 3. Apply the gene mapping to convert from probe level to gene level measurements
print(f"Applying gene mapping with {len(mapping_df)} entries...")
gene_data = apply_gene_mapping(gene_data, mapping_df)

print(f"Converted gene data shape: {gene_data.shape}")
print("First 5 gene symbols after mapping:")
print(gene_data.index[:5].tolist())


Looking for gene symbol information in the SOFT file...


Could not parse platform annotation data
Creating fallback mapping using probe IDs as pseudo-genes
Sample of fallback mapping:
          ID       Gene
0  1007_s_at  1007_s_at
1    1053_at    1053_at
2     117_at     117_at
3     121_at     121_at
4  1255_g_at  1255_g_at
Applying gene mapping with 22268 entries...
Converted gene data shape: (6, 636)
First 5 gene symbols after mapping:
['AFFX-', 'HSAC07', 'HUMGAPDH', 'HUMISGF3A', 'HUMRGE']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index
print("\nNormalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print("First 10 gene identifiers after normalization:")
print(normalized_gene_data.index[:10].tolist())

# Save the normalized gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Check if we have enough valid gene data to proceed
if normalized_gene_data.shape[0] == 0:
    print("No valid gene symbols found after normalization. Dataset is not usable.")
    # Create a minimal DataFrame with the trait column for validation
    dummy_df = pd.DataFrame({trait: []})
    # Conduct final quality validation with appropriate flags
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,  # No valid gene data after normalization
        is_trait_available=False,  # From previous steps, we know trait_row is None
        is_biased=True,  # Explicitly set to True to indicate unusable dataset
        df=dummy_df,
        note="Dataset contains no valid gene symbols after normalization and no trait information for Liver Cancer."
    )
    print("Dataset deemed not usable for associative studies. Linked data not saved.")
else:
    # 2. Link clinical and genetic data
    print("\nLinking clinical and genetic data...")
    # Check if trait data is available
    is_trait_available = trait_row is not None
    
    if not is_trait_available:
        print(f"No trait data available for {trait}. Cannot create linked dataset.")
        # Create a dummy DataFrame with the trait column
        dummy_df = pd.DataFrame({trait: []})
        
        # Conduct final quality validation
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=False,
            is_biased=True,  # Explicitly marked as biased since no trait data available
            df=dummy_df,
            note=f"Dataset contains gene expression data but no information about {trait}."
        )
        print("Dataset deemed not usable for associative studies. Linked data not saved.")
    else:
        # This code won't be reached since trait_row is None, but included for completeness
        linked_data = geo_link_clinical_genetic_data(clinical_data, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview:")
        print(linked_data.iloc[:5, :5])

        # Check and rename the trait column if needed
        if trait not in linked_data.columns and '0.0' in linked_data.columns:
            # Rename the column to the expected trait name
            linked_data = linked_data.rename(columns={'0.0': trait})
            print(f"Renamed column '0.0' to '{trait}'")

        # 3. Handle missing values
        print("\nHandling missing values...")
        linked_data_clean = handle_missing_values(linked_data, trait)
        print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

        # 4. Check for bias in the dataset
        print("\nChecking for bias in dataset features...")
        trait_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

        # 5. Conduct final quality validation and save metadata
        print("\nConducting final quality validation...")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=trait_biased,
            df=linked_data_clean,
            note="Dataset contains gene expression data and trait information."
        )

        # 6. Save the linked data if it's usable
        if is_usable:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            linked_data_clean.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Dataset deemed not usable for associative studies. Linked data not saved.")


Normalizing gene symbols...
Gene data shape after normalization: (0, 636)
First 10 gene identifiers after normalization:
[]


Normalized gene data saved to ../../output/preprocess/Liver_Cancer/gene_data/GSE218438.csv
No valid gene symbols found after normalization. Dataset is not usable.
Abnormality detected in the cohort: GSE218438. Preprocessing failed.
Dataset deemed not usable for associative studies. Linked data not saved.
